In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
%matplotlib inline 
import matplotlib.pyplot as plt

In [3]:
housing = pd.read_csv('Ames_HousePrice.csv', index_col=0)
real_est = pd.read_csv('Ames Real Estate Data.csv')

#Creating a new total bathroom number feature
housing['Baths'] = housing['BsmtFullBath'] + (housing['BsmtHalfBath']/2) + housing['FullBath']+ (housing['HalfBath']/2)

housingdf = housing.copy()


C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
housing.SalePrice.mean()

178059.62325581396

In [72]:
housing.loc[housing.Neighborhood == 'GrnHill']['SalePrice'].mean()

280000.0

In [4]:
housingdf.drop(columns=['PID','Street','Alley','Utilities','Neighborhood','Condition1',
                    'Condition2','BldgType','HouseStyle', 'RoofMatl',
                    'Exterior1st','Exterior2nd','MasVnrType','BsmtExposure',
                    'BsmtFinType1','BsmtFinType2','Heating','1stFlrSF','2ndFlrSF','PoolQC',
                     'MiscFeature','MiscVal','YrSold','MoSold','SaleType','SaleCondition',
                     'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','MSSubClass',
                     'LotConfig','Foundation','GarageYrBlt','Fence'],inplace=True)

In [5]:
quant = [col for col in housingdf.columns if housingdf.dtypes[col] != 'object']
qual = [col for col in housingdf.columns if housingdf.dtypes[col] == 'object']

In [6]:
housingdf[quant] = housingdf.loc[:, quant].fillna(value=0)

In [7]:
h_quant = housingdf.loc[:, quant]

In [8]:
for col in qual:
    housingdf[col] = housingdf[col].astype('category')
    if housingdf[col].isnull().any():
        housingdf[col] = housingdf[col].cat.add_categories(['MISSING'])
        housingdf[col] = housingdf[col].fillna('MISSING')

### Dummifying Nominal Features

In [9]:
nominals = ['MSZoning','Street','LandContour','Utilities','LotConfig','Neighborhood',
           'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
           'Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical',
           'Functional','GarageType', 'PavedDrive', 'SaleType','SaleCondition','LotShape','LandSlope']

In [10]:
noms = [q for q in qual if q in nominals]

In [11]:
nomsdf = housingdf.loc[:, noms]

In [12]:
nomsdummy = pd.get_dummies(nomsdf, drop_first=True)

### Ordinal Encoder for Ordinal Features

In [13]:
ords = [q for q in qual if q not in nominals]

In [14]:
ords

['ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [15]:
ord_df = housingdf.loc[:, ords]

In [16]:
#dictionaries to assign ordinal values
quality = {'MISSING':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
finish = {'MISSING':0,'Unf':1,'RFn':2,'Fin':3}

In [17]:
qc = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC', 'KitchenQual',
      'FireplaceQu','GarageQual','GarageCond']
for c in qc:
    ord_df[c] = ord_df[c].map(quality) 

In [18]:
ord_df['GarageFinish'] = ord_df['GarageFinish'].map(finish)

In [19]:
ord_df

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond
1,3,3,3,3,3,3,4,1,3,3
2,4,3,4,3,3,4,0,3,3,3
3,4,3,3,3,5,4,0,1,3,1
4,4,4,2,3,4,3,0,1,3,3
5,4,3,4,3,5,4,0,3,3,3
...,...,...,...,...,...,...,...,...,...,...
759,3,3,3,3,2,2,4,1,2,1
760,3,3,0,0,5,3,4,1,3,3
761,3,3,3,3,3,3,0,1,3,3
762,4,3,4,3,5,4,3,3,3,3


### Combining Dataframes

In [20]:
cleandropped = pd.concat([h_quant,ord_df,nomsdummy],axis=1)

---

# Regression 

In [21]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

In [52]:
X = cleandropped.drop('SalePrice',axis=1)
y = (cleandropped['SalePrice'])
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [53]:
lcv = LassoCV(normalize=True).fit(X_train,y_train)

In [54]:
print("The train set R^2 is %.4f" %lcv.score(X_train,y_train))
print("The test set R^2 is %.4f" %lcv.score(X_test,y_test))

The train set R^2 is 0.8867
The test set R^2 is 0.8848


In [55]:
lcv.intercept_

-617283.7898857626

In [56]:
coefs = pd.Series(lcv.coef_, index=X.columns).sort_values(ascending=False)

In [58]:
coefs.nlargest(25)

Functional_Typ        22671.789827
LandContour_HLS       20529.915524
GarageType_MISSING    14967.491351
Functional_Min2       14045.840088
Functional_Min1       13658.289057
RoofStyle_Shed        12216.275538
ExterQual             10961.402822
LandContour_Low       10559.496126
GarageType_CarPort    10241.220629
OverallQual           10124.103792
MSZoning_I (all)       9267.580363
LandContour_Lvl        7895.982396
KitchenQual            6856.072154
Functional_Maj2        6656.477846
LandSlope_Mod          5789.152248
OverallCond            5698.463696
Functional_Sal         5605.651804
Fireplaces             5461.915599
LotShape_IR2           4746.906641
BsmtQual               4717.910617
GarageType_BuiltIn     4247.866189
GarageType_Detchd      4038.390626
HeatingQC              2365.487849
MSZoning_FV            2213.448391
GarageCars             1878.461040
dtype: float64

In [59]:
feats = cleandropped[['GrLivArea', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
            'YearBuilt', 'ExterQual', 'BsmtFinSF1', 
            'TotalBsmtSF','KitchenAbvGr', 'KitchenQual','Fireplaces', 'GarageArea', 
            'ScreenPorch','MSZoning_FV',]]

In [60]:
X = feats
y = (cleandropped['SalePrice'])
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [65]:
lcvt = LassoCV(normalize=True).fit(X_train,y_train)
pd.Series(lcvt.coef_, index=X.columns).sort_values(ascending=False)

ExterQual       16344.264008
OverallQual     11643.215763
KitchenQual      9370.672667
Fireplaces       6937.076389
OverallCond      4486.844260
YearBuilt         277.734067
LotFrontage       102.686025
GrLivArea          50.367426
ScreenPorch        42.094876
GarageArea         31.657832
BsmtFinSF1         22.493940
TotalBsmtSF        21.907253
LotArea             0.573052
MSZoning_FV      -446.452133
KitchenAbvGr   -16014.823719
dtype: float64

## np log

In [29]:

features = cleandropped.drop('SalePrice',axis=1)
target = np.log(cleandropped['SalePrice'])
X_train1,X_test1,y_train1,y_test1 = train_test_split(features, target, test_size=0.2,random_state=42)

lcvlog = LassoCV(normalize=True).fit(X_train1, y_train1)

print('-'*50)
print("The train set R^2 is %.4f" %(lcvlog.score(X_train1, y_train1)))
print("The test set R^2 is %.4f" %(lcvlog.score(X_test1, y_test1)))

--------------------------------------------------
The train set R^2 is 0.9068
The test set R^2 is 0.9170


In [30]:
pd.Series(lcvlog.coef_, index=X.columns).sort_values(ascending=False).nlargest(15)

OverallQual          0.071193
CentralAir_Y         0.045982
OverallCond          0.040076
LandSlope_Mod        0.033991
GarageCars           0.028629
Functional_Typ       0.028070
LandContour_HLS      0.026382
KitchenQual          0.023848
Fireplaces           0.023111
ExterQual            0.017789
HeatingQC            0.017288
MSZoning_FV          0.014492
GarageType_Attchd    0.014219
Baths                0.010771
BsmtQual             0.008841
dtype: float64

## dropping columns that have zero effect on sale price

In [31]:
cleanv2 = cleandropped.drop(['Baths','BsmtUnfSF','GarageType_Attchd','Electrical_MISSING',
                  '3SsnPorch','MSZoning_RL','RoofStyle_Hip','FireplaceQu'],axis=1)

In [32]:
X2= cleanv2.drop('SalePrice',axis=1)
y2 = cleanv2['SalePrice']

X2_train,X2_test,y2_train,y2_test = train_test_split(X2, y2, test_size=0.2,random_state=42)

In [33]:
lcv = LassoCV(normalize=True)
lcv.fit(X2_train,y2_train)

LassoCV(normalize=True)

In [34]:
print('train score:',lcv.score(X2_train,y2_train))
print('test score:',lcv.score(X2_test,y2_test))

train score: 0.8867281105602688
test score: 0.8848580399214497


In [35]:
pd.Series(lcv.coef_, index=X2.columns).sort_values(ascending=False).nlargest(25)

Functional_Typ        24192.969431
LandContour_HLS       20822.104203
Functional_Min2       15652.486129
Functional_Min1       15267.866201
GarageType_MISSING    15243.647598
RoofStyle_Shed        12640.749638
LandContour_Low       10960.034162
ExterQual             10925.134708
GarageType_CarPort    10673.214516
OverallQual           10105.232701
MSZoning_I (all)       9462.611206
Functional_Maj2        8448.319794
LandContour_Lvl        8131.219001
Functional_Sal         7914.949272
KitchenQual            6840.666466
LandSlope_Mod          5789.786196
OverallCond            5749.454975
Fireplaces             5467.235863
LotShape_IR2           4801.785696
BsmtQual               4752.533133
GarageType_BuiltIn     4294.561422
GarageType_Detchd      4195.688103
Functional_Mod         3230.983301
HeatingQC              2372.508998
MSZoning_FV            2315.295850
dtype: float64

---

In [36]:
import statsmodels.api as sm 
X_add_const = sm.add_constant(X)
ols = sm.OLS(y, X_add_const)
ans = ols.fit()
print(ans.summary())
print(ans.rsquared)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     272.4
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:12:41   Log-Likelihood:                -29798.
No. Observations:                2580   AIC:                         5.974e+04
Df Residuals:                    2506   BIC:                         6.018e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -6.552e+05   9.83